In [53]:
import numpy as np
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import GurobiOptimizer
from qiskit_optimization.converters import QuadraticProgramToQubo

In [99]:
# parámetros
n_BAT = 1
Ts = 60  # s
C_E = 27000  # kJ
r_FC = 4.7E-8
V_FC = 100
N = 2

# sistema espacio estado discreto
# estado x      estado u
# SOC           P_FC
# H2
# P_M

A = np.array([
    [1, 0, -n_BAT / C_E * Ts],
    [0, 1, 0],
    [0, 0, 1]
])

B = np.array([
    [n_BAT * Ts / C_E],
    [-r_FC * Ts / V_FC],
    [0]
])

n = A.shape[0]  # número de estados
m = B.shape[1]  # número de entradas

# matrices propagación estado
Psi = np.zeros((N * n, n))
for i in range(1, N + 1):
    Psi[n * (i - 1):n * i, :] = np.linalg.matrix_power(A, i)

Upsilon = np.zeros((N * n, m))
for i in range(1, N + 1):
    for j in range(1, i + 1):
        Upsilon[n * (i - 1):n * i, :] += np.linalg.matrix_power(A, j - 1) @ B

Theta = np.zeros((N * n, N * m))
for i in range(1, N + 1):
    for j in range(1, i + 1):
        for k in range(1, i - j + 2):
            Theta[n * (i - 1):n * i, m * (j - 1):m * j] += np.linalg.matrix_power(A, k - 1) @ B

# %% costos
q = np.diag([10, 1, 0])
r = 0
Q = np.kron(np.eye(N), q)
R = np.kron(np.eye(N), r)

# %% referencia
ref = np.array([0.9, 0.9, 0])
T = np.tile(ref, N)

# %% constraints
# input
u_select = np.array([1])
u_max = 128
u_min = 0
F = np.kron(np.tril(np.ones(N)), np.vstack([u_select, -u_select]))
f = np.tile(np.array([u_max, u_min]), N)

# input rate
du_select = np.array([1])
du_max = 15 * Ts
du_min = 15 * Ts
A_du = np.kron(np.eye(N), np.vstack([du_select, -du_select]))
b_du = np.tile(np.array([du_max, du_min]), N).reshape(-1, 1)

# state
x_select = np.array([
    [1, 0, 0],
    [0, 1, 0]
])
x_max = np.array([1, 1])
x_min = np.array([0, 0])
Gamma = np.kron(np.eye(N), np.vstack([x_select, -x_select]))
g = np.tile(np.hstack([x_max, x_min]), N)

# %% simular
# condición inicial
x = np.zeros((3, N))
x[:, 0] = [0.5, 0.5, 100]
u = np.zeros((m, N))
du = np.zeros((m, N))

# optimización offline
H = Theta.T @ Q @ Theta + R
H = (H + H.T) / 2

# optimización online
# **con solo una optimizacion en el tiempo 0, no hay u_(k-1)
k = 0

#epsilon = T - Psi @ x[:, k] - Upsilon @ u[:, k-1]
epsilon = T - Psi @ x[:, k]

G = 2 * Theta.T @ Q @ epsilon

# constraints de input y estado a forma A*x<=b
A_u = F.copy()
#b_u = -F[:, [0]] @ u[:, [k-1]] + f.reshape(-1, 1)
b_u = f.reshape(-1, 1)

A_x = Gamma @ Theta
#b_x = -Gamma @ (Psi @ x[:, [k]] + Upsilon @ u[:, [k-1]]) + g.reshape(-1, 1)
b_x = -Gamma @ (Psi @ x[:, [k]]) + g.reshape(-1, 1)

Ac = np.vstack([A_du, A_u, A_x])
bc = np.vstack([b_du, b_u, b_x])

In [109]:
# problema normal
problem = QuadraticProgram("qp");
problem.continuous_var_list(m*N, -1e20, 1e20, "du");
problem.minimize(linear=-G, quadratic=H);
print(problem.prettyprint())

for i in range(bc.size):
    problem.linear_constraint(Ac[i,:], '<=', bc[i]);

optimizer = GurobiOptimizer(disp=True)
result = optimizer.solve(problem)
result

Problem name: qp

Minimize
  0.0002469135802508898*du0^2 + 0.00019753086420071185*du0*du1
  + 4.938271605017796e-05*du1^2 - 0.10271598170271605*du0
  - 0.037530841637530864*du1

Subject to
  No constraints

  Continuous variables (2)
    du0
    du1

Set parameter NonConvex to value 2
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-11375H @ 3.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
NonConvex  2

Optimize a model with 16 rows, 2 columns and 22 nonzeros
Model fingerprint: 0x5f7a03f1
Model has 3 quadratic objective terms
Coefficient statistics:
  Matrix range     [3e-08, 1e+00]
  Objective range  [4e-02, 1e-01]
  QObjective range [1e-04, 5e-04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e-02, 9e+02]
Presolve removed 14 rows and 0 columns
Presolve time: 0.01s
Presolved: 2 rows, 3 columns, 5 nonzero

<OptimizationResult: fval=-9.102213559049389, du0=127.99999999796908, du1=1.0247731552226469e-09, status=SUCCESS>